### Spuštění PySpark

`pyspark --master yarn --num-executors 2 --executor-memory 4G --packages com.databricks:spark-csv_2.10:1.5.0 --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je váš den a měsíc narození, napr. spark.ui.port=10811

In [ ]:
# uzitecny import
from pyspark.sql import functions as F

### 0. Spark SQL v jednoduchých příkladech

Projděte si soubor s jednoduchými příklady `spark_sql_elementary.ipynb`.

### 1. Načtení dat a zakladní explorace

#### Data

Soubor obsahuje záznamy o písních.

* format: CSV
* cesta: `/user/pascepet/data/lyrics/lyrics.csv` (na HDFS)
* oddelovac: `','`
* header: ne
* pole: `id, nazev, rok, interpret, zanr, text`

1.1 Načtěte soubor do DataFrame (s automatickým odvozením schématu) a přiřaďte sloupcům správné názvy polí (viz výše).  
```
songsDF = sqlContext.read \
	.format("com.databricks.spark.csv") \
	.option("header", "false") \
	.option("delimiter", ",") \
    .option("inferSchema", "true") \
	.load("/user/pascepet/data/lyrics/lyrics.csv")

songsDF = songsDF.toDF('id', 'nazev', 'rok', 'interpret', 'zanr', 'text')
```

1.2 Soubor nakešujte do paměti.  
```
songsDF.cache()
```

1.3 Vypište si ukázku dat.  
```
songsDF.show()
```

1.4 Zjistěte celkový počet záznamů (řádků) v DataFrame. *(362 237)*  
```
songsDF.count()
```

1.5 Zjistěte, kolik písní má jako interpreta Boba Dylana ('bob-dylan'). *(614)*  
```
songsDF.filter(songsDF['interpret']=='bob-dylan').count()
```

1.6 Zjistěte, jaky je nejnižší a nejvyšší uvedený rok písně. *(67, 2038)*  
```
songsDF.groupBy().min('rok').show()
songsDF.groupBy().max('rok').show()
# nebo:
songsDF.select('rok').orderBy('rok').show(1)
songsDF.select('rok').orderBy('rok', ascending=False).show(1)
```

1.7 Vypište žánry v pořadí podle počtu zastoupených písní (od největšího po nejmenší). *(Rock 131 377, Pop 49 444 atd.)*  
```
songsDF.select('zanr') \
    .groupBy('zanr').count() \
    .toDF('zanr', 'pocet') \
    .orderBy('pocet', ascending=False) \
    .show()
```

1.8 Vytvořte si z DataFrame dočasnou tabulku a pokuste se zadání 1.4&ndash;1.7 provést i pomocí SQL.  
```
songsDF.registerTempTable("songs")
sqlContext.sql("select count(*) from songs").show()
sqlContext.sql("select count(*) from songs where interpret='bob-dylan'").show()
sqlContext.sql("select min(rok), max(rok) from songs").show()
sqlContext.sql("select zanr, count(*) as pocet from songs group by zanr order by pocet desc").show()
```

### 2. Úprava a čištění dat

2.1 Vyřaďte všechny záznamy, které mají uvedený rok mimo rozmezí 1950&nbsp;2018. Zjistěte, kolik záznamů v DataFrame zůstalo. *(362 221)*  
```
songsDF2 = songsDF.filter('rok>=1950 and rok<=2018')
songsDF2.count()
```

2.2 Upravte texty písní takto:  
* Text převeďte na malá písmena.
* Všechny nealfanumerické znaky nahraďte mezerou.
* Posloupnosti více mezer nahraďte jedinou mezerou.
* Vypusťte mezery na obou krajích textu (funkce *trim*).  
Podrobně o regulárních výrazech viz [manuál Pythonu](https://docs.python.org/2/library/re.html).  
```
# prevod textu na mala pismena
songsDF2 = songsDF2.withColumn('text', F.lower(songsDF['text']))
# nahrazeni nealfanumerickych znaku mezerou
songsDF2 = songsDF2.withColumn('text', F.regexp_replace(songsDF2['text'], '[\W ]', ' '))
# nahrazeni posloupnosti mezer jedinou mezerou
songsDF2 = songsDF2.withColumn('text', F.regexp_replace(songsDF2['text'], '[ ]+', ' '))
# odstraneni mezer na krajich textu
songsDF2 = songsDF2.withColumn('text', F.trim(songsDF2['text']))
```

2.3 Přidejte do DataFrame sloupec `slova_poc` obsahující počet všech slov písně. Slova jsou v textu oddělena mezerami. Zkontrolujte, zda pro písně s prázdným polem `text` je počet slov 0, a pokud to tak není, opravte v takových případech počet slov na 0.  
```
songsDF2 = songsDF2.withColumn('slova_poc', F.size(F.split(songsDF2['text'], ' ')))
songsDF2.filter('text=""').show()
songsDF2 = songsDF2.withColumn('slova_poc', F.when(songsDF2['text']=='', 0).otherwise(songsDF2['slova_poc']))
```

2.4 Výsledný DataFrame opět nakešujte.  
```
songsDF2.cache()
```

### 3. Analytické dotazy

3.1 Zjistěte, kolik interpretů ma aspoň 500 písní a kteří to jsou. Vytvořte si pro tyto interprety samostatný DataFrame, využijete ho v zadání 4.3. *(19; Bob Dylan 614, Chris Brown 655 atd.)*  
```
interprets = songsDF2.groupBy('interpret').count() \
    .toDF('interpret', 'pocet').filter("pocet >= 500")

interprets.count()
interprets.show()
```

3.2 Vezmeme-li v úvahu jen písně s neprázdným textem (tj. počet slov větší než 0), který z interpretů s aspoň 100 takovými písněmi má nejvyšší průměrný počet slov na píseň? *(bone-thugs-n-harmony 758,05)*  
```
songsDF2.filter('slova_poc > 0') \
    .groupBy('interpret').agg({'*':'count', 'slova_poc':'avg'}) \
    .toDF('interpret', 'prumer', 'pocet').filter('pocet>=100') \
    .orderBy('prumer', ascending=False) \
    .show()
```

### 4. Text-mining

4.1 Najděte 20 nejčastěji se vyskytujících aspoň dvouznakových slov v textech písní. (Každé slovo počítejte tolikrát, kolikrát je v textu uvedeno. Zde se hodí DataFrame zpracovat pomocí RDD transformací.)  
*(('the', 2214416), ('you', 2121404), ('to', 1274821), ('and', 1256593), ..., ('no', 335530))*  
Poté najděte 20 nejčastěji se vyskytujících aspoň dvouznakových slov s vyloučením stop-words. Soubor se stopwords je na HDFS: `/user/pascepet/data/stopwords.txt`  
*(('can', 379061), ('don', 333748), ('love', 324916), ('like', 313862), ..., ('will', 170726))*  
```
words_top = songsDF2.flatMap(lambda r: r[5].split(" ")) \
    .filter(lambda r: len(r)>1) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a,b: a+b) \
    .sortBy(lambda r: r[1], False)
    
words_top.take(20)

# s vypustenim slov patricich do stopwords
stopw = sc.textFile("/user/pascepet/data/stopwords.txt").collect()
stopw = set(stopw)

words_top2 = songsDF2.flatMap(lambda r: r[5].split(" ")) \
    .filter(lambda r: len(r)>1) \
    .filter(lambda r: r not in stopw) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a,b: a+b) \
    .sortBy(lambda r: r[1], False)
    
words_top2.take(20)
```

4.2 Vyberte z množiny nejčastějších slov mimo stop-words tři podle vlastního uvážení. K DataFrame přidejte tři sloupce (pro každé slovo jeden sloupec) s příznakem True/False, zda je v písni dané slovo aspoň jednou uvedeno.  
```
songsDF2 = songsDF2.withColumn('is_love', F.when(F.regexp_extract(songsDF2['text'], r'\b(love)\b', 1) == 'love', 1).otherwise(0))
songsDF2 = songsDF2.withColumn('is_like', F.when(F.regexp_extract(songsDF2['text'], r'\b(like)\b', 1) == 'like', 1).otherwise(0))
songsDF2 = songsDF2.withColumn('is_know', F.when(F.regexp_extract(songsDF2['text'], r'\b(know)\b', 1) == 'know', 1).otherwise(0))
```

4.3 U interpretů s aspoň 500 písněmi (viz 3.1) zjistěte, v jakém podílu jejich písní se vyskytují tři vámi vybraná častá slova ze zadání 4.2.

|        interpret| avg(is_like)| avg(is_know)| avg(is_love)|
|-----------------|-------------|-------------|-------------|
|        bob-dylan| 0.480... | 0.451...| 0.335...|
|      chris-brown| 0.743...| 0.774...| 0.603...|
|       beach-boys| 0.301...| 0.392...| 0.417...|
|  atd. | ... | ... | ... |

```
interprets_words = interprets.join(songsDF2, 'interpret') \
    .select('interpret', 'is_love', 'is_like', 'is_know') \
    .groupBy('interpret') \
    .agg({'is_love':'avg', 'is_like':'avg', 'is_know':'avg'})

interprets_words.show()
```
